# Project for Deep Learning course: Question Answering on Mathematics Dataset

# Task
> Mathematical reasoning – a
core ability within human intelligence – presents some unique challenges as a domain:
we do not come to understand and solve mathematical problems primarily on the back
of experience and evidence, but on the basis of inferring, learning, and exploiting laws,
axioms, and symbol manipulation rules. The task consists of many different types
of mathematics problems, with the motivation that it should be harder for a model
to do well across a range of problem types without possessing at least some part of
these abilities that allow for algebraic generalization.



# Import and installation

Here we install and import all the libraries that we will utilize


In [1]:
!pip install pytorch_lightning transformers datasets wget gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.wh

In [2]:
import json
import pytorch_lightning as pl
import numpy as np
import os
import wget
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer
from transformers import BertTokenizer
from torch.utils.data import DataLoader #random_split
from pytorch_lightning import Trainer, LightningDataModule, LightningModule
from datasets import load_dataset

# Parameters

Here we define all the parameters needed. The most important to choose is the dataset ("num") which allows to change the portion of the math-dataset used, and so to train the model to solve different operations. [num can vary from 0 to 4]

In [3]:
#define parameters
SEED = 42 #random seed
BATCH_SIZE = 32 #batch size
train_samples = 1000 #number of samples for training and validation set (respectively 90 and 10 %)
test_samples = 100 #number of samples for test set
lista_dataset = ["algebra__linear_1d", "arithmetic__add_or_sub", "arithmetic__div", "arithmetic__mul", "arithmetic_simplify_surd"] #list of possible dataset
device = "cuda" if torch.cuda.is_available() else "cpu" #device used

In [4]:
#choose the dataset
num = 0 #from 0 to 4

# Utils

Here we define some functions that will be utilized by each model presented. More in specific we define: a class to convert dataset into Tensors and an Accuracy to evaluate the results during and after the training

In [5]:
class Dataset_class:
    def __init__(self, q, att_q, a, att_a):
        self.input_ids = q
        self.attention_mask = att_q
        self.answer_ids = a
        self.answer_attention_mask = att_a

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, item):
        return [
            torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.attention_mask[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_attention_mask[item], dtype=torch.long).to(device)
        ]


In [6]:
#accuracy written to evaluate the model
def evaluate_accuracy(predicted, target):
    predicted = torch.argmax(predicted, dim=2)

    score_per_char = 0
    total_score = 0
    count = 0
    size_0 = predicted.size(0)
    size_1 = predicted.size(1)

    for i in range(0, size_0):
        score = 0
        for j in range(0, size_1):
            if predicted[i][j].item() == target[i][j].item():
                score += 1
            count += 1
            if target[i][j].item() == 1:
                break
        if score == count:
            total_score += 1
        score_per_char += score

    total_accuracy = total_score / size_0
    total_acc_char = score_per_char / count

    return total_accuracy, total_acc_char


# Load Dataset and tokenization

Here we define some functions to create the dictionaries, codify the dataset with the dictionaries created and create the Dataloaders. At the end we load the partiotion of the dataset chose and randomly seeelect the data needed between the data present in the partition loaded (based on the number of samples required).

In [7]:
def build_dict(data):
    char_to_idx = {}
    idx_to_char = {}
    all_characters = set()
    question_max_length = 0
    answer_max_length = 0

    for split_name in ["train", "test"]:
        split_dataset = data[split_name]

        for row in split_dataset:
            question = row['question']
            answer = row['answer']
            question_length = len(row['question'])
            answer_length = len(row['answer'])

            all_characters.update(question)
            all_characters.update(answer)

            if question_length > question_max_length:
                question_max_length = question_length

            if answer_length > answer_max_length:
                answer_max_length = answer_length

    char_idx = 0  # Initialize a variable to keep track of character indices

    # Add special tokens
    special_tokens = ['<start>', '<end>', 'PAD', 'UNK']

    for token in special_tokens:
        char_to_idx[token] = char_idx
        idx_to_char[char_idx] = token
        char_idx += 1  # Increment the index for the next special token

    # Assign each character a unique index

    for char in all_characters:
        char_to_idx[char] = char_idx
        idx_to_char[char_idx] = char
        char_idx += 1  # Increment the index for the next character

    return char_to_idx, idx_to_char, question_max_length, answer_max_length


In [8]:
def Tokenize(data, char_to_idx, max_length_q, max_length_a):
    max_length_q -= 3
    padded_question = []
    attention_mask_q = []
    for string in data["question"]:
        numerical_indices_q = [char_to_idx['<start>']] + [char_to_idx[char] if char in char_to_idx else char_to_idx['UNK'] for char in string][2:len(string) - 3] + [char_to_idx['<end>']]
        if len(numerical_indices_q) < max_length_q:
            numerical_indices_q += [char_to_idx['PAD']] * (max_length_q - len(numerical_indices_q))
        padded_question.append(numerical_indices_q)
        att = [1 for _ in range(0, len(string) - 3)] + [0 for _ in range(0, max_length_q - (len(string) - 3))]
        attention_mask_q.append(att)

    max_length_a -= 3
    padded_answer = []
    attention_mask_a = []
    for string in data["answer"]:
        numerical_indices_a = [char_to_idx['<start>']] + [char_to_idx[char] if char in char_to_idx else char_to_idx['UNK'] for char in string][2:len(string) - 3] + [char_to_idx['<end>']]
        if len(numerical_indices_a) < max_length_a:
            numerical_indices_a += [char_to_idx['PAD']] * (max_length_a - len(numerical_indices_a))
        padded_answer.append(numerical_indices_a)
        att = [1 for _ in range(0, len(string) - 3)] + [0 for _ in range(0, max_length_a - (len(string) - 3))]
        attention_mask_a.append(att)

    return Dataset_class(padded_question, attention_mask_q, padded_answer, attention_mask_a)


In [9]:
def create_dataset(data, batch_size):
    char_to_idx, idx_to_char, max_length_q, max_length_a = build_dict(data)
    traindataset = Tokenize(data['train'][:int(len(data['train']) * 0.9)], char_to_idx, max_length_q, max_length_a)
    vaaliddataset = Tokenize(data['train'][int(len(data['train']) * 0.9):], char_to_idx, max_length_q, max_length_a)
    testdataset = Tokenize(data['test'], char_to_idx, max_length_q, max_length_a)

    train_dataloaders = torch.utils.data.DataLoader(traindataset, shuffle=True, batch_size=batch_size)
    valid_dataloaders = torch.utils.data.DataLoader(vaaliddataset, shuffle=False, batch_size=batch_size)
    test_dataloaders = torch.utils.data.DataLoader(testdataset, shuffle=False, batch_size=batch_size)

    return train_dataloaders, valid_dataloaders, test_dataloaders, [char_to_idx, idx_to_char, max_length_q - 3, max_length_a - 3]


In [10]:
from datasets import DatasetDict
# Load the dataset from HuggingFace
dataset = load_dataset('math_dataset', lista_dataset[num])

# Resize the dataset on the portion needed
dataset = DatasetDict({
    "train": dataset["train"].shuffle(seed=SEED).select(range(train_samples)),
    "test": dataset["test"].shuffle(seed=SEED).select(range(test_samples))
})

# Create the dataloaders and save the parameters needed for the model
train_dataloaders, valid_dataloaders, test_dataloaders, params = create_dataset(dataset, BATCH_SIZE)


Generating train split:   0%|          | 0/1999998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
import pickle
with open("/paramas.pkl", 'wb') as f:
    pickle.dump(params, f)

# Basile model: LSTM

This is a baseline model to evaluate better the performance of the proposed model (Our solution). This is a very basic solution that employes an lstm to predict the answer

## Model

In [12]:
class LSTMmodel(LightningModule):
    def __init__(self, char_to_idx, idx_to_char, question_max_length=68, answer_max_length=10, batch_size=128,
                 embedding_size=512, hidden_units=2048, num_encoder_layers=6, padding_idx=2, p=0.):
        super().__init__()

        self.pad_token = "PAD"
        self.dict_size = len(char_to_idx)
        self.question_max_length = question_max_length
        self.answer_max_length = answer_max_length
        self.padding_idx = padding_idx
        self.idx_to_char = idx_to_char
        self.dropout_probability = p
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_encoder_layers = num_encoder_layers

        self.embedding = nn.Embedding(self.dict_size, self.embedding_size, padding_idx=self.padding_idx)
        self.lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_units,
                            num_layers=self.num_encoder_layers, dropout=self.dropout_probability, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_units, self.dict_size)
        self.list_loss = []
        self.list_accuracy = []
    def forward(self, batch):
        input_tensor = batch[0]
        embedded = self.embedding(input_tensor)
        h0 = torch.zeros(self.num_encoder_layers, embedded.size(0), self.hidden_units).to(embedded.device)
        c0 = torch.zeros(self.num_encoder_layers, embedded.size(0), self.hidden_units).to(embedded.device)
        lstm_out, _ = self.lstm(embedded, (h0, c0))
        sequence_length = batch[2].size(1)
        lstm_out = lstm_out[:, :sequence_length, :]
        logits = self.output_layer(lstm_out)

        return logits

    def training_step(self, batch, _):
        batch_answers = batch[2][:, 1:].flatten(0, 1)
        batch[2] = batch[2][:, :-1]
        pred = self(batch).flatten(0, 1)

        loss = F.cross_entropy(pred, batch_answers, ignore_index=self.padding_idx)
        print(loss.item())
        self.list_loss.append(loss.item())
        return loss

    def validation_step(self, batch, _):
        batch_answers = batch[2]
        pred = self.predict(batch)

        accuracy = evaluate_accuracy(pred, batch_answers)
        print("val_tot_accuracy", accuracy)
        self.list_accuracy.append(accuracy)
        return accuracy

    def test_step(self, batch, _):
        batch_answers = batch[2]
        pred = self.predict(batch.copy())

        accuracy = evaluate_accuracy(pred, batch_answers)
        print("test_tot_accuracy", accuracy)
        return accuracy

    def predict(self, batch):
        batch[2] = torch.tensor([[1] for j in range(len(batch[2]))], device=device).long()

        for i in range(self.answer_max_length-1):
            transformer_result = self(batch)
            batch[2] = torch.tensor([], device=device).long()

            for j in range(len(transformer_result)):
                predicted_chars = torch.argmax(transformer_result[j], 1)

                if len(predicted_chars.shape) == 1:
                    predicted_chars = predicted_chars.unsqueeze(0)

                start_line_char = torch.tensor([1], device=device).unsqueeze(0).long()
                predicted_chars = torch.cat((start_line_char, predicted_chars), 1)
                batch[2] = torch.cat((batch[2], predicted_chars), 0)

        return F.one_hot(batch[2], num_classes=self.dict_size)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))
    def get_loss(self):
        # Questo metodo restituirà il valore di self.data
        return self.list_loss

In [13]:
#Define the model
model_LSTM = LSTMmodel(params[0], params[1], params[2], params[3], BATCH_SIZE)

## Training


In [ ]:
#Train the model
trainer = pl.Trainer(max_epochs=30)
#Train the model using the train_dataloaders
trainer.fit(model_LSTM, train_dataloaders, valid_dataloaders)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type      | Params
-------------------------------------------
0 | embedding    | Embedding | 25.1 K
1 | lstm         | LSTM      | 188 M 
2 | output_layer | Linear    | 100 K 
-------------------------------------------
188 M     Trainable params
0         Non-trainable params
188 M     Total params
755.870   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


val_tot_accuracy (0.0, 0.0)
val_tot_accuracy (0.0, 0.0)


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

val_tot_accuracy (0.0, 0.36363636363636365)
val_tot_accuracy (0.0, 0.38235294117647056)
val_tot_accuracy (0.0, 0.35714285714285715)
val_tot_accuracy (0.0, 0.3333333333333333)
1.6663438081741333
1.7911977767944336
1.7343666553497314
1.709606409072876
1.6431143283843994
1.7283122539520264
1.77446711063385
1.7981071472167969
1.6423132419586182
1.687936544418335
1.6739435195922852
1.6592034101486206
1.724266529083252
1.7953318357467651
1.7309556007385254
1.7600703239440918
1.6485469341278076
1.7507280111312866
1.719417691230774
1.8026560544967651
1.618649959564209
1.7704342603683472
1.6720045804977417
1.7633750438690186
1.697740077972412
1.728188157081604
1.7393819093704224
1.6290329694747925
1.817699909210205


Validation: |          | 0/? [00:00<?, ?it/s]

val_tot_accuracy (0.0, 0.36363636363636365)
val_tot_accuracy (0.0, 0.38235294117647056)
val_tot_accuracy (0.0, 0.35714285714285715)
val_tot_accuracy (0.0, 0.3333333333333333)


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
#Save the model
torch.save(model_LSTM.state_dict(), "/content/drive/MyDrive/NewCheckpoints/model_checkpoint_LSTM10.pth")

RuntimeError: ignored

# State of the art: TP Transformar

## model


This is the state of the art model for this task right now. It is made with a Transformer architecture and based on SelfAttention mechanism

In [ ]:
class TotalEmbeddings(LightningModule):
    def __init__(self, dict_size, sizes, embedding_size, padding_idx=2, p=0.1):
        super().__init__()

        self.dict_size = dict_size
        self.max_length = max(sizes[0], sizes[1])
        self.embedding_size = embedding_size
        self.padding_idx = padding_idx

        self.dropout_probability = p
        self.scale = math.sqrt(self.embedding_size)
        self.embedding_layer = nn.Embedding(self.dict_size, self.embedding_size, padding_idx=self.padding_idx)
        self.dropout = nn.Dropout(p=self.dropout_probability)

    def forward(self, input):
        input = input.to(self.embedding_layer.weight.device)
        char_embedding = self.embedding_layer(input) * self.scale

        # Sinusoidal Positional Embeddings Computation
        positional_embedding = self._initialize_pos_embedding(char_embedding.size(1)).to(char_embedding.device)

        return self.dropout(char_embedding + positional_embedding)

    def _initialize_pos_embedding(self, max_len):
        positional_embeddings = torch.zeros(max_len, self.embedding_size, device=self.embedding_layer.weight.device)

        for pos in range(max_len):
            for i in range(0, self.embedding_size, 2):
                positional_embeddings[pos, i] = math.sin(pos / (10000 ** (i / self.embedding_size)))
                positional_embeddings[pos, i + 1] = math.cos(pos / (10000 ** (i / self.embedding_size)))

        return positional_embeddings


In [ ]:
class PositionwiseFFN(LightningModule):
    def __init__(self, embedding_size, hidden_units):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units

        self.linear_1 = nn.Linear(self.embedding_size, self.hidden_units)
        self.linear_2 = nn.Linear(self.hidden_units, self.embedding_size)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.linear_1(input)
        output = self.relu(output)
        output = self.linear_2(output)
        return output


In [ ]:
class Decoder(LightningModule):

    def __init__(self, embedding_size, hidden_units, num_layers, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout_probability = p
        self.p_attention = p_AT

        self.layers = nn.ModuleList([DecoderLayer(self.embedding_size,  self.hidden_units, p_AT= self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

    def forward(self, encoder_batch, outputs_batch, src_padding_mask, tgt_padding_mask):
        decoder_layer_output = outputs_batch

        for decoder_layer in self.layers:
            decoder_layer_output = decoder_layer(encoder_batch, decoder_layer_output, src_padding_mask, tgt_padding_mask)

        return decoder_layer_output


In [ ]:
class DecoderLayer(LightningModule):

    def __init__(self, embedding_size, hidden_units, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.dropout_probability = p
        self.p_attention = p_AT

        self.ffn = PositionwiseFFN(self.embedding_size, self.hidden_units)
        self.selfAttn = SelfAttention(self.p_attention)
        self.layer_norm_1 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_2 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_3 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_4 = nn.LayerNorm(self.embedding_size, device=device)
        self.densefilter = PositionwiseFeedforward(hid_dim=self.p_attention.d_x,
                                                   pf_dim=self.p_attention.d_r,
                                                   dropout=self.p_attention.dropout)
        self.dropout = nn.Dropout(p=self.dropout_probability)

    def forward(self, src, trg, src_mask, trg_mask):
        z = self.layer_norm_2(trg)
        z = self.selfAttn(z, z, z, trg_mask)
        z = self.dropout(z)
        trg = trg + z

        # Encoder attention
        z = self.layer_norm_2(trg)
        z = self.selfAttn(z, src, src, src_mask)
        z = self.dropout(z)
        trg = trg + z

        # Dense filter
        z = self.layer_norm_3(trg)
        z = self.densefilter(z)
        z = self.dropout(z)
        trg = trg + z

        return self.layer_norm_4(trg)


In [ ]:
class Encoder(LightningModule):

    def __init__(self, embedding_size, hidden_units, num_layers, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout_probability = p
        self.p_attention = p_AT

        self.layers = nn.ModuleList([EncoderLayer(self.embedding_size, self.hidden_units, self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

    def forward(self, batch, src_padding_mask):

        encoder_layer_output = batch

        for encoder_layer in self.layers:
            encoder_layer_output = encoder_layer(encoder_layer_output, src_padding_mask)

        return encoder_layer_output


In [ ]:
class EncoderLayer(LightningModule):

    def __init__(self, embedding_size, hidden_units, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.dropout_probability = p
        self.p_attention = p_AT

        self.selfAttn = SelfAttention(self.p_attention)
        self.layer_norm_1 = nn.LayerNorm(self.embedding_size)
        self.dense_filter = PositionwiseFeedforward(self.embedding_size, self.hidden_units, self.dropout_probability)
        self.layer_norm_2 = nn.LayerNorm(self.embedding_size)
        self.dropout = nn.Dropout(p=self.dropout_probability)
        self.layer_norm_3 = nn.LayerNorm(self.embedding_size)

    def forward(self, src, src_mask):
        # Sublayer 1
        z = self.layer_norm_1(src)
        z = self.selfAttn(z, z, z, src_mask)
        z = self.dropout(z)
        src = src + z

        # Sublayer 2
        z = self.layer_norm_2(src)
        z = self.dense_filter(z)
        z = self.dropout(z)
        src = src + z

        return self.layer_norm_3(src)


In [ ]:
class PositionwiseFeedforward(LightningModule):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.pf_dim = pf_dim

        self.linear1 = nn.Linear(hid_dim, pf_dim)
        self.linear2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)

        self.reset_parameters()

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(F.relu(x))
        x = self.linear2(x)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)


In [ ]:
class SelfAttention(pl.LightningModule):

    def __init__(self, p):
        super().__init__()

        self.p = p
        self.d_h = p.d_x
        self.n_I = p.n_I

        self.W_q = nn.Linear(self.d_h, p.d_q * p.n_I)
        self.W_k = nn.Linear(self.d_h, p.d_k * p.n_I)
        self.W_v = nn.Linear(self.d_h, p.d_v * p.n_I)
        self.W_r = nn.Linear(self.d_h, p.d_r * p.n_I)

        self.W_o = nn.Linear(p.d_v * p.n_I, p.d_x)

        self.dropout = nn.Dropout(p.dropout)
        self.dot_scale = torch.FloatTensor([math.sqrt(p.d_k)])
        self.mul_scale = torch.FloatTensor([1.0 / math.sqrt(math.sqrt(2) - 1)])

    def forward(self, query, key, value, mask=None):
        bsz = query.shape[0]

        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)
        R = self.W_r(query)

        Q = Q.view(bsz, -1, self.n_I, self.p.d_q).permute(0, 2, 1, 3)
        K = K.view(bsz, -1, self.n_I, self.p.d_k).permute(0, 2, 1, 3)
        V = V.view(bsz, -1, self.n_I, self.p.d_v).permute(0, 2, 1, 3)
        R = R.view(bsz, -1, self.n_I, self.p.d_r).permute(0, 2, 1, 3)
        dot = torch.einsum('bhid,bhjd->bhij', Q, K) / self.dot_scale.to(key.device)

        attention = self.dropout(F.softmax(dot, dim=-1))

        v_bar = torch.einsum('bhjd,bhij->bhid', V, attention)

        new_v = v_bar * R
        new_v = new_v.permute(0, 2, 1, 3).contiguous()

        new_v = new_v.view(bsz, -1, self.n_I * self.p.d_v)

        x = self.W_o(new_v)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W_q.weight)
        nn.init.xavier_uniform_(self.W_k.weight)
        nn.init.xavier_uniform_(self.W_v.weight)
        nn.init.xavier_uniform_(self.W_o.weight)
        nn.init.normal_(self.W_r.weight, mean=0, std=1.0 / math.sqrt(self.p.d_r))


In [ ]:
class SelfAttentionConfig:
    def __init__(self, d_x, d_q, d_k, d_v, d_r, dropout, n_I):
        self.d_x = d_x
        self.d_q = d_q
        self.d_k = d_k
        self.d_v = d_v
        self.d_r = d_r
        self.dropout = dropout
        self.n_I = n_I


In [ ]:
class TP_Transformer_SelfAttention(LightningModule):
    def __init__(self, char_to_idx, idx_to_char, question_max_length=68, answer_max_length=10, batch_size=128,
                 embedding_size=512, hidden_units=2048, num_heads=8, num_encoder_layers=6, num_decoder_layers=6,
                 padding_idx=2, p=0.):
        super().__init__()

        self.pad_token = "PAD"
        self.dict_size = len(char_to_idx)
        self.question_max_length = question_max_length
        self.answer_max_length = answer_max_length
        self.padding_idx = padding_idx
        self.idx_to_char = idx_to_char
        self.dropout_probability = p
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.p_attention = SelfAttentionConfig(d_x=512, d_q=64, d_k=64, d_v=64, d_r=64, dropout=self.dropout_probability, n_I=8)

        self.total_embedding = TotalEmbeddings(self.dict_size, [self.question_max_length, self.answer_max_length],
                                               self.embedding_size, self.padding_idx, p=self.dropout_probability)
        self.encoder = Encoder(self.embedding_size, hidden_units=self.hidden_units, num_layers=self.num_encoder_layers,
                               p=self.dropout_probability, p_AT=self.p_attention)
        self.decoder = Decoder(self.embedding_size, hidden_units=self.hidden_units, num_layers=self.num_decoder_layers,
                               p=self.dropout_probability, p_AT=self.p_attention)

    def forward(self, batch):
        questions = (batch[0]).to(device)
        answers = (batch[2]).to(device)

        source_pad_mask = (batch[1]).to(device)
        target_pad_mask = (batch[3]).to(device)

        embedded_questions = self.total_embedding(questions)
        embedded_answers = self.total_embedding(answers)
        encoder_output = self.encoder(embedded_questions, src_padding_mask=source_pad_mask)
        decoder_output = self.decoder(encoder_output, embedded_answers, src_padding_mask=source_pad_mask,
                                       tgt_padding_mask=target_pad_mask)
        return torch.matmul(decoder_output, torch.transpose(self.total_embedding.embedding_layer.weight, 0, 1))

    def training_step(self, batch, _):
        batch_answers = batch[2][:, 1:].flatten(0, 1)
        batch[2] = batch[2][:, :-1]
        pred = self(batch).flatten(0, 1)

        loss = F.cross_entropy(pred, batch_answers, ignore_index=self.padding_idx)
        return loss

    def validation_step(self, batch, _):
        batch_answers = batch[2]
        pred = self.predict(batch)

        accuracy = evaluate_accuracy(pred, batch_answers)
        print("Validation accuracy: ", accuracy)
        return accuracy

    def test_step(self, batch, _):
        batch_answers = batch[2]
        pred = self.predict(batch.copy())

        accuracy = evaluate_accuracy(pred, batch_answers)
        print("Test accuracy: ", accuracy)
        return accuracy

    def predict(self, batch):
        batch[2] = torch.tensor([[1] for j in range(len(batch[2]))], device=device).long()

        for i in range(self.answer_max_length - 1):
            transformer_result = self(batch)
            batch[2] = torch.tensor([], device=device).long()

            for j in range(len(transformer_result)):
                predicted_chars = torch.argmax(transformer_result[j], 1)

                if len(predicted_chars.shape) == 1:
                    predicted_chars = predicted_chars.unsqueeze(0)

                start_line_char = torch.tensor([1], device=device).unsqueeze(0).long()
                predicted_chars = torch.cat((start_line_char, predicted_chars), 1)

                batch[2] = torch.cat((batch[2], predicted_chars), 0)

        return F.one_hot(batch[2], num_classes=self.dict_size)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))



In [ ]:
#Define the model
model_TP = TP_Transformer_SelfAttention(params[0], params[1], params[2], params[3], BATCH_SIZE)

## Training


In [ ]:
#Train the model
trainer = pl.Trainer(max_epochs=30)
#Train the model using the train_dataloaders
trainer.fit(model_TP, train_dataloaders, valid_dataloaders)

In [ ]:
#Save the model
torch.save(model_TP.state_dict(), "/content/drive/MyDrive/NewCheckpoints/model_checkpoint_TPnew4.pth")

# Our solution: Transformer with Hybrid attention

This is the state of the art model for this task right now. It is made with a Transformer architecture and based on SelfAttention mechanism

## Model

In [ ]:
class TotalEmbeddings(LightningModule):
    def __init__(self, dict_size, sizes, embedding_size, padding_idx=2, p=0.1):
        super().__init__()

        self.dict_size = dict_size
        self.max_length = max(sizes[0], sizes[1])
        self.embedding_size = embedding_size
        self.padding_idx = padding_idx
        self.dropout_probability = p

        self.scale = math.sqrt(self.embedding_size)
        self.embedding_layer = nn.Embedding(self.dict_size, self.embedding_size, padding_idx=self.padding_idx)
        self.dropout = nn.Dropout(p=self.dropout_probability)

    def forward(self, input):
        input = input.to(self.embedding_layer.weight.device)
        char_embedding = self.embedding_layer(input) * self.scale

        # Sinusoidal Positional Embeddings Computation
        positional_embedding = self._initialize_pos_embedding(char_embedding.size(1)).to(char_embedding.device)

        return self.dropout(char_embedding + positional_embedding)

    def _initialize_pos_embedding(self, max_len):
        positional_embeddings = torch.zeros(max_len, self.embedding_size, device=self.embedding_layer.weight.device)

        for pos in range(max_len):

            for i in range(0, self.embedding_size, 2):
                positional_embeddings[pos, i] = math.sin(pos / (10000 ** (i / self.embedding_size)))
                positional_embeddings[pos, i + 1] = math.cos(pos / (10000 ** (i / self.embedding_size)))

        return positional_embeddings


In [ ]:
class PositionwiseFFN(LightningModule):
    def __init__(self, embedding_size, hidden_units):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units

        self.linear_1 = nn.Linear(self.embedding_size, self.hidden_units)
        self.linear_2 = nn.Linear(self.hidden_units, self.embedding_size)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.linear_1(input)
        output = self.relu(output)
        output = self.linear_2(output)
        return output


In [ ]:
class Decoder(LightningModule):
    def __init__(self, embedding_size, hidden_units, num_layers, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout_probability = p
        self.p_attention = p_AT

        self.layers = nn.ModuleList([DecoderLayer(self.embedding_size, self.hidden_units, p_AT=self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

    def forward(self, encoder_batch, outputs_batch, src_padding_mask, tgt_padding_mask):
        decoder_layer_output = outputs_batch

        for decoder_layer in self.layers:
            decoder_layer_output = decoder_layer(encoder_batch, decoder_layer_output, src_padding_mask, tgt_padding_mask)

        return decoder_layer_output


In [ ]:
class DecoderLayer(LightningModule):
    def __init__(self, embedding_size, hidden_units, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.dropout_probability = p
        self.p_attention = p_AT

        self.ffn = PositionwiseFFN(self.embedding_size, self.hidden_units)
        self.selfAttn = SelfAttention(self.p_attention)
        self.layer_norm_1 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_2 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_3 = nn.LayerNorm(self.embedding_size, device=device)
        self.layer_norm_4 = nn.LayerNorm(self.embedding_size, device=device)
        self.densefilter = PositionwiseFeedforward(hid_dim=self.p_attention.d_x,
                                                   pf_dim=self.p_attention.d_r,
                                                   dropout=self.p_attention.dropout)
        self.dropout = nn.Dropout(p=self.dropout_probability)

    def forward(self, src, trg, src_mask, trg_mask):
        z = self.layer_norm_2(trg)
        z = self.selfAttn(z, z, z, trg_mask)
        z = self.dropout(z)
        trg = trg + z
        z = self.layer_norm_2(trg)
        z = self.selfAttn(z, src, src, src_mask)
        z = self.dropout(z)
        trg = trg + z
        z = self.layer_norm_3(trg)
        z = self.densefilter(z)
        z = self.dropout(z)
        trg = trg + z

        return self.layer_norm_4(trg)


In [ ]:
class Encoder(LightningModule):
    def __init__(self, embedding_size, hidden_units, num_layers, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout_probability = p
        self.p_attention = p_AT

        self.layers = nn.ModuleList([EncoderLayer(self.embedding_size, self.hidden_units, self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

    def forward(self, batch, src_padding_mask):
        encoder_layer_output = batch

        for encoder_layer in self.layers:
            encoder_layer_output = encoder_layer(encoder_layer_output, src_padding_mask)

        return encoder_layer_output


In [ ]:
class EncoderLayer(LightningModule):
    def __init__(self, embedding_size, hidden_units, p_AT, p=0.1):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.dropout_probability = p
        self.p_attention = p_AT

        self.selfAttn = HybridAttention(self.p_attention, self.embedding_size)
        self.layer_norm_1 = nn.LayerNorm(self.embedding_size)
        self.dense_filter = PositionwiseFeedforward(self.embedding_size, self.hidden_units, self.dropout_probability)
        self.layer_norm_2 = nn.LayerNorm(self.embedding_size)
        self.dropout = nn.Dropout(p=self.dropout_probability)
        self.layer_norm_3 = nn.LayerNorm(self.embedding_size)

    def forward(self, src, src_mask):

        z = self.layer_norm_1(src)
        z = self.selfAttn(z, z, z, src_mask)
        z = self.dropout(z)
        src = src + z
        z = self.layer_norm_2(src)
        z = self.dense_filter(z)
        z = self.dropout(z)
        src = src + z

        return self.layer_norm_3(src)


In [ ]:
class PositionwiseFeedforward(LightningModule):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.pf_dim = pf_dim

        self.linear1 = nn.Linear(hid_dim, pf_dim)
        self.linear2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)

        self.reset_parameters()

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(F.relu(x))
        x = self.linear2(x)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)


In [ ]:
class HybridAttention(pl.LightningModule):
    def __init__(self, attention, embedding_size, num_heads=8, masked=False):
        super().__init__()

        self.embedding_size = embedding_size
        self.num_heads = num_heads
        attention.d_x
        self.key_embedding_size = attention.d_x
        self.masked = masked
        self.dk = self.embedding_size // self.num_heads

        # Initialize the MultiHeadAttention and the new attention mechanism
        self.multi_head_attention = MultiHeadAttention(self.embedding_size, self.num_heads, self.key_embedding_size, self.masked)
        self.new_attention_mechanism = AdditiveAttention(self.embedding_size, self.key_embedding_size)

        # Learnable parameter for weighted combination
        self.alpha = nn.Parameter(torch.tensor(0.5))

    def forward(self, batch_keys, batch_queries, batch_values, trg_mask):
        # Calculate attention from the MultiHeadAttention
        multi_head_attention_result = self.multi_head_attention(batch_keys, batch_queries, batch_values, trg_mask)

        # Calculate attention from the new attention mechanism
        new_attention_result = self.new_attention_mechanism(batch_keys, batch_queries, batch_values, trg_mask)

        # Combine the results with a learnable parameter to weight them
        hybrid_attention_result = self.alpha * multi_head_attention_result + (1 - self.alpha) * new_attention_result.sum(dim=1, keepdim=True)

        return hybrid_attention_result


class MultiHeadAttention(pl.LightningModule):
    def __init__(self, embedding_size, num_heads, key_embedding_size, masked=False):
        super().__init__()

        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.key_embedding_size = key_embedding_size
        self.masked = masked
        self.dk = self.embedding_size // self.num_heads

        self.attention_heads = nn.ModuleList([AttentionHead(self.dk, self.embedding_size, self.key_embedding_size, self.masked) for _ in range(self.num_heads)])

    def forward(self, batch_keys, batch_queries, batch_values, padding_mask):
        attention_heads_results = []
        for head in self.attention_heads:
            attention_heads_results.append(head(batch_keys, batch_queries, batch_values, padding_mask))

        # Sum the results from different attention heads
        return torch.sum(torch.stack(attention_heads_results), dim=0)



class AttentionHead(pl.LightningModule):
    def __init__(self, dk, embedding_size, key_embedding_size, masked):
        super().__init__()

        self.dk = dk
        self.masked = masked
        self.embedding_size = embedding_size
        self.key_embedding_size = key_embedding_size

        self.Wk = nn.Linear(key_embedding_size, dk, bias=False)
        self.Wq = nn.Linear(embedding_size, dk, bias=False)
        self.Wv = nn.Linear(embedding_size, dk, bias=False)
        self.Wr = nn.Linear(embedding_size, dk, bias=False)
        self.Wo = nn.Linear(dk, embedding_size, bias=False)

    def forward(self, batch_keys, batch_queries, batch_values, padding_mask):
        # Project keys, queries, and values
        keys = self.Wk(batch_keys)
        queries = self.Wq(batch_queries)
        values = self.Wv(batch_values)
        roles = self.Wr(batch_queries)
        # Calculate unnormalized attention scores
        unnormalized_attention_score = torch.matmul(queries, keys.transpose(1, 2)) / math.sqrt(self.dk)

        if self.masked:
            input_len_1 = batch_queries.shape[1]
            input_len_2 = batch_keys.shape[1]
            target_mask = torch.triu(torch.full((input_len_1, input_len_2), float("-inf"), device=device), diagonal=1)
            unnormalized_attention_score = unnormalized_attention_score + target_mask

        # Apply padding mask
        padding_mask = padding_mask.bool()
        padding_mask = padding_mask.unsqueeze(1).repeat(1, batch_queries.shape[1], 1)
        unnormalized_attention_score = unnormalized_attention_score.masked_fill_(padding_mask, float("-inf"))
        attention_score = F.softmax(unnormalized_attention_score, 2)
        filler = torch.matmul(attention_score, values)

        # Apply the output linear layer with role interaction
        return self.Wo(filler * roles)


class AdditiveAttention(pl.LightningModule):
    def __init__(self, query_embedding_size, key_embedding_size):
        super().__init__()

        self.query_embedding_size = query_embedding_size
        self.key_embedding_size = key_embedding_size

        self.Wq = nn.Linear(query_embedding_size, query_embedding_size)
        self.Wk = nn.Linear(key_embedding_size, key_embedding_size)
        self.V = nn.Linear(query_embedding_size, 1)

    def forward(self, batch_keys, batch_queries, batch_values, padding_mask):
        # Project queries and keys to the same embedding size
        queries = self.Wq(batch_queries)
        keys = self.Wk(batch_keys)
        additive_scores = self.V(torch.tanh(queries + keys)).squeeze(dim=-1)
        if padding_mask is not None:
            padding_mask = padding_mask.bool()
            additive_scores = additive_scores.masked_fill(padding_mask, float("-inf"))

        # Compute attention scores and the weighted sum of values
        attention_scores = F.softmax(additive_scores / math.sqrt(self.query_embedding_size), dim=-1)
        weighted_sum = torch.matmul(attention_scores, batch_values)

        return weighted_sum


In [ ]:
class SelfAttention(pl.LightningModule):
    def __init__(self, p):
        super().__init__()

        self.p = p
        self.d_h = p.d_x
        self.n_I = p.n_I

        self.W_q = nn.Linear(self.d_h, p.d_q * p.n_I)
        self.W_k = nn.Linear(self.d_h, p.d_k * p.n_I)
        self.W_v = nn.Linear(self.d_h, p.d_v * p.n_I)
        self.W_r = nn.Linear(self.d_h, p.d_r * p.n_I)

        self.W_o = nn.Linear(p.d_v * p.n_I, p.d_x)

        self.dropout = nn.Dropout(p.dropout)
        self.dot_scale = torch.FloatTensor([math.sqrt(p.d_k)])
        self.mul_scale = torch.FloatTensor([1.0 / math.sqrt(math.sqrt(2) - 1)])

    def forward(self, query, key, value, mask=None):
        bsz = query.shape[0]

        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)
        R = self.W_r(query)

        Q = Q.view(bsz, -1, self.n_I, self.p.d_q).permute(0, 2, 1, 3)
        K = K.view(bsz, -1, self.n_I, self.p.d_k).permute(0, 2, 1, 3)
        V = V.view(bsz, -1, self.n_I, self.p.d_v).permute(0, 2, 1, 3)
        R = R.view(bsz, -1, self.n_I, self.p.d_r).permute(0, 2, 1, 3)
        dot = torch.einsum('bhid,bhjd->bhij', Q, K) / self.dot_scale.to(key.device)
        attention = self.dropout(F.softmax(dot, dim=-1))
        v_bar = torch.einsum('bhjd,bhij->bhid', V, attention)
        new_v = v_bar * R
        new_v = new_v.permute(0, 2, 1, 3).contiguous()

        new_v = new_v.view(bsz, -1, self.n_I * self.p.d_v)

        x = self.W_o(new_v)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W_q.weight)
        nn.init.xavier_uniform_(self.W_k.weight)
        nn.init.xavier_uniform_(self.W_v.weight)
        nn.init.xavier_uniform_(self.W_o.weight)
        nn.init.normal_(self.W_r.weight, mean=0, std=1.0 / math.sqrt(self.p.d_r))


In [ ]:
class SelfAttentionConfig:
    def __init__(self, d_x, d_q, d_k, d_v, d_r, dropout, n_I):
        self.d_x = d_x
        self.d_q = d_q
        self.d_k = d_k
        self.d_v = d_v
        self.d_r = d_r
        self.dropout = dropout
        self.n_I = n_I


In [ ]:
class Transformer(LightningModule):
    def __init__(self, char_to_idx, idx_to_char, question_max_length=68, answer_max_length=10, batch_size=128,
                 embedding_size=512, hidden_units=2048, num_heads=8, num_encoder_layers=6, num_decoder_layers=6,
                 padding_idx=2, p=0.):
        super().__init__()

        self.pad_token = "PAD"
        self.dict_size = len(char_to_idx)
        self.question_max_length = question_max_length
        self.answer_max_length = answer_max_length
        self.padding_idx = padding_idx
        self.idx_to_char = idx_to_char
        self.dropout_probability = p
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.p_attention = SelfAttentionConfig(d_x=512, d_q=64, d_k=64, d_v=64, d_r=64, dropout=self.dropout_probability, n_I=8)

        self.total_embedding = TotalEmbeddings(self.dict_size, [self.question_max_length, self.answer_max_length],
                                               self.embedding_size, self.padding_idx, p=self.dropout_probability)
        self.encoder = Encoder(self.embedding_size, hidden_units=self.hidden_units, num_layers=self.num_encoder_layers,
                               p=self.dropout_probability, p_AT=self.p_attention)
        self.decoder = Decoder(self.embedding_size, hidden_units=self.hidden_units, num_layers=self.num_decoder_layers,
                               p=self.dropout_probability, p_AT=self.p_attention)

    def forward(self, batch):
        questions = (batch[0]).to(device)
        answers = (batch[2]).to(device)

        source_pad_mask = (batch[1]).to(device)
        target_pad_mask = (batch[3]).to(device)

        embedded_questions = self.total_embedding(questions)

        embedded_answers = self.total_embedding(answers)

        encoder_output = self.encoder(embedded_questions, src_padding_mask=source_pad_mask)

        decoder_output = self.decoder(encoder_output, embedded_answers, src_padding_mask=source_pad_mask,
                                       tgt_padding_mask=target_pad_mask)

        return torch.matmul(decoder_output, torch.transpose(self.total_embedding.embedding_layer.weight, 0, 1))

    def training_step(self, batch, _):
        batch_answers = batch[2][:, 1:].flatten(0, 1)
        batch[2] = batch[2][:, :-1]
        pred = self(batch).flatten(0, 1)

        loss = F.cross_entropy(pred, batch_answers, ignore_index=self.padding_idx)
        return loss

    def validation_step(self, batch, _):
        batch_answers = batch[2]

        pred = self.predict(batch)
        print(pred)
        accuracy = evaluate_accuracy(pred, batch_answers)
        print("Validation accuracy: ", accuracy)
        return accuracy

    def test_step(self, batch, _):
        batch_answers = batch[2]
        pred = self.predict(batch.copy())

        accuracy = evaluate_accuracy(pred, batch_answers)
        print("Test accuracy: ", accuracy)
        return accuracy

    def predict(self, batch):
        batch[2] = torch.tensor([[1] for j in range(len(batch[2]))], device=device).long()

        for i in range(self.answer_max_length - 1):
            transformer_result = self(batch)
            batch[2] = torch.tensor([], device=device).long()

            for j in range(len(transformer_result)):
                predicted_chars = torch.argmax(transformer_result[j], 1)

                if len(predicted_chars.shape) == 1:
                    predicted_chars = predicted_chars.unsqueeze(0)

                start_line_char = torch.tensor([1], device=device).unsqueeze(0).long()
                predicted_chars = torch.cat((start_line_char, predicted_chars), 1)

                batch[2] = torch.cat((batch[2], predicted_chars), 0)

        return F.one_hot(batch[2], num_classes=self.dict_size)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))



In [ ]:
#Define the model
model = Transformer(params[0], params[1], params[2], params[3], BATCH_SIZE)

## Training

In [ ]:
#Train the model
trainer = pl.Trainer(max_epochs=30)
#Train the model using the train_dataloaders
trainer.fit(model, train_dataloaders, valid_dataloaders)

In [ ]:
#Save the model
torch.save(model.state_dict(), "/content/drive/MyDrive/NewCheckpoints/model_checkpoint_TP2.pth")

# Download pretrained weights

In [ ]:
!gdown 1-3N-VGHo4sjJxqFXwDGMWe_mSsF2hHk_ -O /content/lstm_baseline.ckpt11
!gdown 1-GknyRsaGTSFgUvxjIJ2qMP1jNcY6ta9 -O /content/lstm_baseline_params.pkl1
#!gdown 1-JMz-uwBTGg7obOkJiW8m9hGH9TzfnPX -O /content/lstm_baseline.ckpt12

#!gdown 1-1ZTuDRNO4XW_ZCHZpcu5Qf1zgrPxRtn -O /content/lstm_baseline_params.pkl3
#!gdown 1-04MIfMdRN56lbOMy4B-J-OeD50gR1-S -O /content/lstm_baseline.ckpt32

In [ ]:
#!gdown 1-VobZkuYbLgju0zs0ehJ6o9apXb7X6BW -O /content/transformer_our_model.ckpt
#!gdown 1-wjp76oP-YQMBowpMlC5G2c4fFLGl6rC -O /content/transformer_our_model_params.pkl1

!gdown 1-SgDaJEY_vl3UnSk5GM1-UE5-NTVln17 -O /content/transformer_our_model_params.pkl2
!gdown 1-SL8qHMdrCXHtAmS9L3dIw5JxY_1N6u0 -O /content/transformer_our_model.ckpt2

#!gdown 1-7C16v8WQggSvZEIYO-n-mOxDHiAli5S -O /content/transformer_our_model_params.pkl3
#!gdown 1-4A-x2guysHk8Jq3Rg1qq4wXZLJbVOzk -O /content/transformer_our_model.ckpt3

In [ ]:
#!gdown 1-4A-x2guysHk8Jq3Rg1qq4wXZLJbVOzk -O /content/tp_transformer.ckpt1
#!gdown 1-7C16v8WQggSvZEIYO-n-mOxDHiAli5S -O /content/tp_transformer_params.pkl1
#!gdown 1-LIma3I3RgwEIZyjevez90fC5MJquWo- -O /content/tp_transformer_params.pkl2f
#!gdown 1-AkKDtd25uPV4JVODz2AwGVan4t9f6wq -O /content/tp_transformer.ckpt21
#!gdown 1-IOVAF8xa8qy6H-KDvzAm1-ligamv5Ae -O /content/tp_transformer.ckpt22

#!gdown 1-jvpEDLcdt8ul8Euk3hFmt0AiabSp_-b -O /content/tp_transformer.ckpt31
#!gdown 1-608w9B9BZIMiP9PGsHd-8BxNbvWFon6 -O /content/tp_transformer_params.pkl3
#!gdown 1-DX3uG5LJd-LipedxGO4R-JD9HE-re_ls -O /content/tp_transformer.ckpt32


!gdown 1-LIma3I3RgwEIZyjevez90fC5MJquWo- -O /content/tp_transformer_params.pkl12


!gdown 1-AkKDtd25uPV4JVODz2AwGVan4t9f6wq -O /content/tp_transformer_checkpoint.pkl13
#!gdown 1-IOVAF8xa8qy6H-KDvzAm1-ligamv5Ae -O /content/tp_transformer_checkpoint.pkl14
#!gdown 1-SL8qHMdrCXHtAmS9L3dIw5JxY_1N6u0 -O /content/tp_transformer_checkpoint.pkl15

# Testing


In [ ]:
# Choose your model
choice = 2
trainer = pl.Trainer(max_epochs=10)
import pickle
params = None
if choice == 1:
    with open("/content/lstm_baseline_params.pkl1", 'rb') as file:
        params = pickle.load(file)
    model_LSTM.load_state_dict(torch.load("/content/lstm_baseline.ckpt11"), strict=False)
    model_LSTM.eval()
    trainer.test(model_LSTM, test_dataloaders)
elif choice == 2:
    with open("/content/tp_transformer_params.pkl12", 'rb') as file:
        params = pickle.load(file)
    model_TP.load_state_dict(torch.load("/content/tp_transformer_checkpoint.pkl13"), strict=False)
    model_TP.eval()
    trainer.test(model_TP, test_dataloaders)
elif choice == 3:
    with open("/content/transformer_our_model_params.pkl3", 'rb') as file:
        params = pickle.load(file)
    model.load_state_dict(torch.load("/content/transformer_our_model.ckpt3"), strict=False)
    model.eval()
    trainer.test(model, test_dataloaders)
else:
    print("No model selected")




# Prediction

Before running the code it is necessary to previously run the section where the selected model is defined

In [ ]:
choice = 2
trainer = pl.Trainer(max_epochs=10)
import pickle
params = None
if choice == 1:
    with open("/content/lstm_baseline_params.pkl1", 'rb') as file:
        params = pickle.load(file)
    model_LSTM.load_state_dict(torch.load("/content/lstm_baseline.ckpt11"), strict=False)
    model_LSTM.eval()
elif choice == 2:
    with open("/content/tp_transformer_params.pkl12", 'rb') as file:
        params = pickle.load(file)
    model_TP.load_state_dict(torch.load("/content/tp_transformer_checkpoint.pkl13"), strict=False)
    model_TP.eval()
elif choice == 3:
    with open("/content/transformer_our_model_params.pkl3", 'rb') as file:
        params = pickle.load(file)
    model.load_state_dict(torch.load("/content/transformer_our_model.ckpt3"), strict=False)
    model.eval()
else:
    print("No model selected")

### Prediction functions


In [ ]:
def tokenize( input,char_to_idx,question_max_length):
  question = []
  question.append(1)  # adding start of line char

  for char in input:
    if char not in char_to_idx.keys():
      question.append(1)
    else:
      question.append(char_to_idx[char])  # turning character into index

  question.append(2)  # adding end of line char

  for i in range(question_max_length-len(input)-2):
      question.append(0)  # adding padding after

  question = torch.tensor(question, device=device).unsqueeze(0)

  return {"question": question}

In [ ]:
import numpy as np

def create_attention_mask(input_tensor):
    """
    Create a fully connected attention mask for a given tensor.

    Args:
        input_tensor (numpy.ndarray): The input tensor.

    Returns:
        numpy.ndarray: The attention mask with 1s indicating positions to attend to.
    """
    N, M = input_tensor.shape
    attention_mask = np.ones((N, M), dtype=int)
    return attention_mask


In [ ]:
def encode_question(inputs1,inputs2,inputs3,inputs4):

    tesst = Dataset_class3(inputs1,inputs2,inputs3,inputs4)
    dataloaders = torch.utils.data.DataLoader(tesst, shuffle=False, batch_size=16)

    for batch in dataloaders:
      pred = model_LSTM.predict(batch.copy())
    predicted = torch.argmax(pred, dim=2)

    return predicted


In [ ]:
def tensor_to_string(tensor, idx_to_char):
    # Extract the elements from the tensor and map them to characters
    char_list = [idx_to_char[idx.item()] for idx in tensor.squeeze()]
    # Join the characters to form the string
    return ''.join(char_list)




In [ ]:
class Dataset_class3:
  def __init__(self, inputs1,inputs2,inputs3,inputs4):
    self.input_ids = inputs1
    self.attention_mask = inputs2
    self.answer_ids = inputs3
    self.answer_attention_mask = inputs4

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, item):
    return [torch.tensor(self.input_ids[item], dtype=torch.long),
            torch.tensor(self.attention_mask[item], dtype=torch.long),
            torch.tensor(self.answer_ids[item], dtype=torch.long),
            torch.tensor(self.answer_attention_mask[item], dtype=torch.long)]

In [ ]:
def answerquestion(question):


  string_question = tokenize( question,params[0],params[2])
  string_answer = tokenize( "b'-19\n'",params[0],params[3])
  TensorPredictedAnswer = encode_question(string_question["question"],create_attention_mask(string_question["question"]),string_answer["question"],create_attention_mask(string_answer["question"]))
  decoded_answer = tensor_to_string(TensorPredictedAnswer, params[1])
  return decoded_answer

In [ ]:
Example_inthe_dataset = 22
test = dataset["test"][Example_inthe_dataset]

In [ ]:
string_question = tokenize( test["question"],params[0],params[2])
string_answer = tokenize( test["answer"],params[0],params[3])


In [ ]:
TensorPredictedAnswer = encode_question(string_question["question"],create_attention_mask(string_question["question"]),string_answer["question"],create_attention_mask(string_answer["question"]))

In [ ]:
# Translate the tensors to strings
decoded_answer = tensor_to_string(TensorPredictedAnswer, params[1])

### Preditction

In [ ]:
print("Question:",dataset["test"]["question"][Example_inthe_dataset])
print("Real answer:",dataset["test"]["answer"][Example_inthe_dataset])
print("Predicted answer:",decoded_answer)

In [ ]:
user_question = input("Type equation: ")
variable = input("For which varible?: ")
user_question_modifed = "b'Solve "+user_question+" for "+variable+ ".\n'"

answer = answerquestion(user_question_modifed)
print("The answer to your question is",answer)